<a href="https://colab.research.google.com/github/cxctis/Google-Colab-Experiment/blob/main/Whisper_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import files
import os

# 1. Upload
print("Please upload your file:")
uploaded = files.upload()
# FIX: It should be .keys() with an 's'
file_name = list(uploaded.keys())[0]

# 2. Convert to MP3
# FIX: We use !ffmpeg (command line) because the 'ffmpeg' python library
# often requires complex setup in Colab. This is much more reliable:
audio_output = "audio_converted.mp3"
!ffmpeg -i "{file_name}" -acodec libmp3lame -ar 44100 -ac 2 -ab 192k "{audio_output}" -y

# 3. Load Whisper
import whisper
# Note: Ensure you ran !pip install openai-whisper in a previous cell
model = whisper.load_model("medium")
print(f"Transcribing {file_name}... this may take a few minutes.")

# 4. Transcribe
# Added fp16=False to prevent warnings if you aren't using a GPU
result = model.transcribe(audio_output, language="en", fp16=False)

print("\n---DONE!---\n")
print(result["text"])

# 5. Save
with open("transcription.txt", "w") as f:
    f.write(result["text"])

print("\nYour transcription has been saved as 'transcription.txt' in the files folder on the left.")

Please upload your file:


Saving The-Knowledge-Centre-on-Interpretation_Media.mp4 to The-Knowledge-Centre-on-Interpretation_Media.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable

In [10]:
import datetime

import whisper
model = whisper.load_model("medium")
print(f"Transcribing with timestamp...this might take a moment.")

result = model.transcribe(audio_output, language="en", fp16=False)

def format_timestamp(seconds: float):
  td = datetime.timedelta(seconds=seconds)
  total_seconds = int(td.total_seconds())
  hours, reminder = divmod(total_seconds, 3600)
  minutes, seconds_int = divmod(reminder, 60)
  milliseconds = int((seconds -int(seconds)) * 1000)
  return f"{hours:02}:{minutes:02}:{seconds_int:02},{milliseconds:03}"

caption_text = ""
for i, segment in enumerate(result['segments'], start=1):
  start = format_timestamp(segment['start'])
  end = format_timestamp(segment['end'])
  text = segment['text'].strip()

  caption_line = f"{i}\n{start} --> {end}\n{text}\n\n"
  caption_text += caption_line

print("\n--- CAPTION PREVIEW ---\n")
print(caption_text[:500] + "...") # Preview the first 500 chars

with open("captions.srt", "w", encoding="utf-8") as f:
    f.write(caption_text)

print("\nSuccess! Your 'captions.srt' file is ready in the left folder.")

from google.colab import files
files.download('captions.srt')

Transcribing with timestamp...this might take a moment.

--- CAPTION PREVIEW ---

1
00:00:00,000 --> 00:00:08,080
Hello, my name is Cathy Pearson and I'm the project manager for Skic's new flagship project,

2
00:00:08,080 --> 00:00:11,800
the Knowledge Centre on Interpretation.

3
00:00:11,800 --> 00:00:19,000
You may have seen some of our promotional videos on Twitter or Facebook, but I would

4
00:00:19,000 --> 00:00:22,719
like to tell you a little bit more about this project.

5
00:00:22,719 --> 00:00:25,160
First of all, what is it?

6
00:00:25,320 --> 00:00:32,320
Well...

Success! Your 'captions.srt' file is ready in the left folder.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!pip install git+https://github.com/openai/whisper.git yt-dlp

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-_vpl8uhz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-_vpl8uhz
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.7 MB/s eta 0:00:00


In [15]:
# 1. INSTALL NECESSARY TOOLS
!pip install git+https://github.com/openai/whisper.git yt-dlp
!sudo apt update && sudo apt install ffmpeg

import yt_dlp
import whisper
import datetime
from google.colab import files

# 2. INPUT THE URL
video_url = input("Enter the YouTube Video URL: ")

# 3. DOWNLOAD AUDIO ONLY
print("Downloading audio from YouTube...")
ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': 'youtube_audio.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

audio_file = "youtube_audio.mp3"

# 4. TRANSCRIBE
print("Loading model and transcribing...")
model = whisper.load_model("medium")
result = model.transcribe(audio_file, fp16=True) # Set to True for GPU speed

# 5. FORMAT AS CAPTIONS (SRT)
def format_timestamp(seconds: float):
    td = datetime.timedelta(seconds=seconds)
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds_int = divmod(remainder, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{seconds_int:02},{milliseconds:03}"

caption_text = ""
for i, segment in enumerate(result['segments'], start=1):
    start = format_timestamp(segment['start'])
    end = format_timestamp(segment['end'])
    text = segment['text'].strip()
    caption_text += f"{i}\n{start} --> {end}\n{text}\n\n"

# 6. SAVE AND DOWNLOAD
file_output = "youtube_transcript.srt"
with open(file_output, "w", encoding="utf-8") as f:
    f.write(caption_text)

print(f"\n--- DONE! ---\nFile saved as {file_output}")
files.download(file_output)

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-urxhjvgw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-urxhjvgw
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launch

[youtube] F9_diBeynPM: Downloading android vr player API JSON
[info] F9_diBeynPM: Downloading 1 format(s): 140
[download] Destination: youtube_audio.m4a
[download] 100% of    1.32MiB in 00:00:00 at 2.15MiB/s   
[FixupM4a] Correcting container of "youtube_audio.m4a"
[ExtractAudio] Destination: youtube_audio.mp3
Deleting original file youtube_audio.m4a (pass -k to keep)
Loading model and transcribing...

--- DONE! ---
File saved as youtube_transcript.srt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>